## My little database JSON-LD import

#### Import relevant packages

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing
import bw_migrations

import os
import numpy as np
import pandas as pd

#### Create/set the working folder:

In [2]:
bd.projects.dir

PosixPath('/Users/mmendez/Library/Application Support/Brightway3/default.c21f969b5f03d33d43e04f8f136e7682')

In [3]:
bd.projects.set_current('MLD_JSON_2')

### Debugging units

Here I solely extract the whole database without any changes, the JSONImporter 

In [4]:
from bw2io.extractors.json_ld import JSONLDExtractor
from collections import Counter

In [5]:
extractor = JSONLDExtractor

In [6]:
path0 = '../7_BW2_Importing_USLCI/databases/My_little_database' # For some reason if I use the relative path it throws me an error
data = JSONLDExtractor.extract(path0)

#### Import JSON-LD LCI:

In [7]:
path = '../7_BW2_Importing_USLCI/databases/My_little_database' # For some reason if I use the relative path it throws me an error
mld = bi.importers.JSONLDImporter(
    path, 
    "My_little_dataset_v0", 
    preferred_allocation="PHYSICAL_ALLOCATION"
)

#### Apply strategies to map JSON-LD to Brightway2 schema:

In [8]:
mld.apply_strategies()

Applying strategy: json_ld_allocate_datasets
Applying strategy: json_ld_get_normalized_exchange_locations
Applying strategy: json_ld_convert_unit_to_reference_unit
Applying strategy: json_ld_get_activities_list_from_rawdata
Applying strategy: json_ld_add_products_as_activities
Applying strategy: json_ld_get_normalized_exchange_units
Applying strategy: json_ld_add_activity_unit
Applying strategy: json_ld_rename_metadata_fields
Applying strategy: json_ld_location_name
Applying strategy: json_ld_remove_fields
Applying strategy: json_ld_fix_process_type
Applying strategy: json_ld_label_exchange_type
Applying strategy: json_ld_prepare_exchange_fields_for_linking
Applying strategy: add_database_name
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: normalize_units
Applied 17 strategies in 0.00 seconds

	Created 3 biosphere flows in separate database 'My_little_dataset_v0 biosphere'.
	Use either `.merge_biosphere_flows()` or `.write_separ

#### Check database dictionaries:

In [9]:
bd.databases # As expected, nothing was written.

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

#### Write the biosphere database:

In [10]:
mld.write_separate_biosphere_database()

Writing activities to SQLite3 database:
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/21/2022 15:17:27
  Finished: 09/21/2022 15:17:27
  Total time elapsed: 00:00:00
  CPU %: 2.50
  Memory %: 0.49
Created database: My_little_dataset_v0 biosphere


In [11]:
bd.databases

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

#### Write the technosphere database:

In [12]:
bio = bd.Database('My_little_dataset_v0 biosphere')

In [13]:
mld.write_database()

Writing activities to SQLite3 database:


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/21/2022 15:17:27
  Finished: 09/21/2022 15:17:27
  Total time elapsed: 00:00:00
  CPU %: 2.50
  Memory %: 0.49
Created database: My_little_dataset_v0


Brightway2 SQLiteBackend: My_little_dataset_v0

<div class="alert alert-block alert-warning">
<b>To fix:</b>
    <li>The importer does not recognize the geolocations.</li>
    <li>The importer does not import the LCIA methods. <i>Hint:</i> Check bw2setup() function, this one loads LCIAs and biosphere flows.</li>
</div>

In [14]:
bd.databases

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

In [15]:
db = bd.Database('My_little_dataset_v0')

#### Let's import the methods

In [16]:
mld_methods = bi.importers.JSONLDLCIAImporter(path)

In [17]:
mld_methods.apply_strategy(bi.strategies.json_ld_lcia.json_ld_lcia_add_method_metadata)

Applying strategy: json_ld_lcia_add_method_metadata


In [18]:
mld_methods.apply_strategy(bi.strategies.json_ld_lcia.json_ld_lcia_set_method_metadata)

Applying strategy: json_ld_lcia_set_method_metadata


In [19]:
mld_methods.apply_strategy(bi.strategies.json_ld_lcia.json_ld_lcia_convert_to_list)

Applying strategy: json_ld_lcia_convert_to_list


In [20]:
mld_methods.data

dict_values([{'@context': 'http://greendelta.github.io/olca-schema/context.jsonld', '@type': 'ImpactCategory', '@id': '6ff74fb6-3d8e-4c2e-ba7e-64e2fd4376c4', 'name': 'Global warming', 'version': '00.00.000', 'referenceUnitName': 'kg CO2 eq', 'impactFactors': [{'@type': 'ImpactFactor', 'value': 2.0, 'flow': {'@type': 'Flow', '@id': 'cd154d8f-0694-43b2-b4ab-e44101e122bd', 'name': 'Bad stuff', 'categoryPath': ['Elementary flows', 'emissions'], 'flowType': 'ELEMENTARY_FLOW', 'location': 'DJ', 'refUnit': 'kg'}, 'unit': {'@type': 'Unit', '@id': '027023c8-b17c-42f9-babe-893b69d48254', 'name': 'kg'}, 'flowProperty': {'@type': 'FlowProperty', '@id': '65d0b9d1-3295-4e2d-8b1c-71ef05c71fe2', 'name': 'Mass'}}], 'filename': '/Users/mmendez/Documents/Postdoc/Software_dev/Brightway/BW_Tutorials/learningbrightway/7_BW2_Importing_USLCI/databases/My_little_database/lcia_categories/6ff74fb6-3d8e-4c2e-ba7e-64e2fd4376c4.json', 'parent': {'name': 'Mac IAM', 'description': '', 'version': '00.00.001', 'lastCha

In [21]:
# codes = {o["code"] for o in bio}
# print(len(codes))
# for method in mld_methods.data:
#     for cf in method['impactFactors']:
#         if cf["flow"]["@id"] in codes:
#             cf["input"] = (bio, cf["flow"]["@id"])
#             print(cf["input"])
            


In [22]:
mld_methods.match_biosphere_by_id('My_little_dataset_v0 biosphere')

Used


In [23]:
asdfasdfa

NameError: name 'asdfasdfa' is not defined

#### Let's look at the biosphere flows:

In [24]:
[act for act in bio]

['Crude oil' (, None, ('Elementary flows', 'resources')),
 'Bad stuff' (, None, ('Elementary flows', 'emissions')),
 'Virgin metals' (, None, ('Elementary flows', 'resources'))]

#### And now let's look at the our processes and products:

In [25]:
[(act.as_dict(), act['unit']) for act in bio]

[({'code': '8849be54-1b13-4d7e-85f6-2297817333f2',
   'name': 'Crude oil',
   'categories': ('Elementary flows', 'resources'),
   'CAS number': None,
   'database': 'My_little_dataset_v0 biosphere',
   'unit': '',
   'type': 'emission',
   'id': 83},
  ''),
 ({'code': 'cd154d8f-0694-43b2-b4ab-e44101e122bd',
   'name': 'Bad stuff',
   'categories': ('Elementary flows', 'emissions'),
   'CAS number': None,
   'database': 'My_little_dataset_v0 biosphere',
   'unit': '',
   'type': 'emission',
   'id': 84},
  ''),
 ({'code': '7528830a-7344-43be-b484-ac4dc625f272',
   'name': 'Virgin metals',
   'categories': ('Elementary flows', 'resources'),
   'CAS number': None,
   'database': 'My_little_dataset_v0 biosphere',
   'unit': '',
   'type': 'emission',
   'id': 82},
  '')]

In [26]:
[(act, act['type'], act['unit']) for act in db]

[('Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')),
  'product',
  ''),
 ('Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')), 'product', ''),
 ('Bottle assembly' (number_of_items, Djibouti, None),
  'process',
  'number_of_items'),
 ('Stainless steel manufacturing' (kilogram, Djibouti, None),
  'process',
  'kilogram'),
 ('Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')),
  'product',
  ''),
 ('Plastic manufacturing' (kilogram, Djibouti, None), 'process', 'kilogram')]

In [27]:
[(act['name'],[exc for exc in act.exchanges()]) for act in db if act['type'] == 'process']

[('Stainless steel manufacturing',
  [Exchange: 1.0  'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 1.0  'Virgin metals' (, None, ('Elementary flows', 'resources')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 3.0  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>]),
 ('Plastic manufacturing',
  [Exchange: 1.0  'Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Plastic manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 2.0  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Plastic manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 1.0  'Crude oil' (, None, ('Elementary flows', 'resources')) to 'Plastic manufacturing' (kilogram, Djibouti, None)>]),
 ('Bottle assembly',
  [Exchange: 1.0  'Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')) to 

In [28]:
[(act['name'],[exc for exc in act.exchanges()]) for act in db if act['type'] == 'process']

[('Stainless steel manufacturing',
  [Exchange: 1.0  'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 1.0  'Virgin metals' (, None, ('Elementary flows', 'resources')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 3.0  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>]),
 ('Bottle assembly',
  [Exchange: 1.0  'Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.25  'Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.1  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.3  'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle a

#### Which ones are processes and which ones are products? Let's also get their codes:

In [29]:
[(act['name'], act['type'], act['code']) for act in db]

[('Stainless steel manufacturing',
  'process',
  '7e5ec332-09fd-4706-8373-3f140a539028'),
 ('Plastic', 'product', '62714200-1a0d-43fb-9b48-99df3f233c94'),
 ('Bottle assembly', 'process', 'fae6799b-7326-452c-92b3-76758bbcac22'),
 ('Bottle', 'product', 'b806c2cd-d563-43c4-a0c9-9c7dd5d513d3'),
 ('Stainless steel', 'product', '3355a1b9-8fbf-40de-b449-ea6399a8a323'),
 ('Plastic manufacturing', 'process', '2fc6deea-6437-4b9f-bf91-89bca44d30f0')]

### Let's manually add a LCIA method
This importer does not recognize the LCIA methods, therefore we need to add them manually. I will debug this later!

First, let's find the `Bad stuff` biosphere flow.

In [30]:
bad_flow = [act for act in bio if act['name'] == 'Bad stuff'][0]
bad_flow.as_dict()

{'code': 'cd154d8f-0694-43b2-b4ab-e44101e122bd',
 'name': 'Bad stuff',
 'categories': ('Elementary flows', 'emissions'),
 'CAS number': None,
 'database': 'My_little_dataset_v0 biosphere',
 'unit': '',
 'type': 'emission',
 'id': 84}

In [31]:
myLCIAdata = [[(bad_flow['database'], bad_flow['code']), 2.0]] # A method list needs: a reference to the flow: tuple (database, 'code')), a characterization factor number, and localization (if no localization is given, 'GLO' is used)
method_key = ('MacIM', 'Global warming', 'total')
my_method = bd.Method(method_key)
my_method.validate(myLCIAdata)
my_method.register()
my_method.write(myLCIAdata)

#### Now we define a functional unit:
This one might be a bit counterintuitive, our functional unit here is **Impact of assembling 5 bottles**, intuintively one would select the activity, but bw2 selects the flow coming out of the `Bottle assembly` activity (i.e. `Bottle`, which is a `product` not a `process`).

In [32]:
activity = [act for act in db if act['name'] == 'Bottle'][0]

In [33]:
functional_unit = {activity : 5} #Impact of 5 bottles

#### Run the LCA!

In [34]:
lca = bc.LCA(functional_unit, method_key) 

In [35]:
len(dir(lca))

72

In [36]:
lca.lci()   # Builds matrices, solves the system, generates an LCI matrix.

In [37]:
len(dir(lca))

79

In [38]:
lca.lcia()  # Characterization, i.e. the multiplication of the elements  
            # of the LCI matrix with characterization factors from the chosen method


In [39]:
len(dir(lca))

82

In [40]:
lca.score    # Returns the score, i.e. the sum of the characterized inventory

15.00000037252903

Scoooooooore!!!